In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/home/joe/testbed/DeepSurvivalMachines')

In [3]:
from dsm import datasets
x, t, e = datasets.load_dataset('PBC', sequential = True)

In [4]:
import numpy as np
import seaborn as sns
import pandas as pd

In [7]:
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F

# reusing compression code via gumbel

In [5]:
import utils

In [8]:
bs = 128 # batch size

In [9]:
torch.cuda.is_available()

True

In [14]:
x_train, y_train, x_val, y_val = utils.prepocess_training_data_chirag_from_list(x, t, vsize=0.25, random_state=563)

AttributeError: module 'utils' has no attribute 'prepocess_training_data_chirag_from_list'